## Anatomy of a Spark Application
The main entry point into a Spark application is what is known as the Spark Context. The Spark Context represents the connection to the Spark cluster, and is what we use to define our Spark job. It's also what Spark uses to create Resilient Distributed Datasets (RDDs). We import the SparkContext from the pyspark library and denote the context with the variable name `sc`

In [ ]:
from pyspark import SparkConf, SparkContext
from tests import Lab01_Tests

In [ ]:
tests = Lab01_Tests()

In [ ]:
sc

Note the Spark UI. This will become an exceedingly important dashboard to inspect and lives at localhost:4040.

### Getting Acquainted with the Spark UI
1. Visit http://localhost:4040
2. Answer the below questions and check your answers by running the appropriate cells. 

#### Question 1: What is the default scheduler Spark is using?

In [ ]:
scheduler = 'ANSWER_HERE'

In [ ]:
tests.part_a(scheduler)

#### Question 2: What version of Spark is the VM running?

In [ ]:
version = 'ANSWER_HERE'

In [ ]:
tests.part_b(version)

#### Question 3: Where are the event logs located?

In [ ]:
path = 'ANSWER_HERE'

In [ ]:
tests.part_c(path)

## Map, Filter, and Reduce
RDD's implement three transformations that you'll want to be familiar with; map(), filter(), and reduce(). Suppose I wanted to take a given string, and count up the number of occurences of each letter. It's a trivial example, but it illustrates the core of how these transformations work in Spark.  

### Map
The map() transformation takes a given RDD and transforms it into another RDD using some custom code that we as the developer specify. First, let's create an RDD:

In [ ]:
message = 'The rain in Spain falls mainly on the plain.'
rdd = sc.parallelize(list(message))
print(rdd)

We create RDD's using the parrallelize() method. This tells the Spark Context to create a distributed dataset that can be operated on in parallel. Let's map the inputs

In [ ]:
tokens = rdd.map(lambda x: (x, 1))
print(tokens)

Suppose we wanted to take a look at what's inside the RDD? We need to be careful here. Since the RDD is distributed, we need to collect its contents and return them back to a single machine one element at a time. This is generally a very slow operation, and could potentially result in an Out of Memory exception. For this small RDD, it's fine.

In [ ]:
tokens.collect()

But there's a problem. We have some extra characters we probably don't want, namely space and period. How do we deal with these?

### Filter
Recall the caveat we had with collect(). If the RDD is large, then the driver program might crash. We can filter down to subset of data using the filter() transformation. Let's use the filter() transformation to create an RDD that doesn't contain any special characters.

In [ ]:
filtered_tokens = tokens.filter(lambda token: token[0] is not ' ' and token[0] is not '.')

In [ ]:
filtered_tokens.collect()

We're now ready to compute our letter count. To do this, we'll turn to the reduceByKey() transformation.

### Reduce
The reduceByKey() transformation takes a pair of key value pairs, and calculates a reduction that we define. In this case, the reduction is going to reduce each character pair down to a single key containing the letter, and then a sum of occurences of that letter.

In [ ]:
result = filtered_tokens.reduceByKey(lambda x, y: x + y)

In [ ]:
result.sortBy(lambda x: x[1], ascending=False).collect()

And that's it! Only not quite. We still have a bug. Note that S, s, T, and t are all counted as separate characters. That isn't quite what we're looking for. Let's fix this.

### Fix the RDD
1. Map the the buggy RDD to a new RDD that contains only lower case characters
2. Reduce and sort the new RDD from highest occurence to lowest occurence
3. Collect the results
4. Run the Test Cell

In [ ]:
fixed_rdd = 'ANSWER_HERE' # Write a single line of the form result.map().reduceByKey().sortBy().collect()

In [ ]:
tests.part_d(fixed_rdd)

### File I/O
Up to this point we've been working with a single string. This; however, isn't a good use case for Spark since a single string easily fits in main memory. Using Spark's built in textFile() method, we can read in one or more plain text files.

In [2]:
text_file_rdd = sc.textFile('data/long_file.txt')

The above command reads in the works of William Shakespeare. Like before, the result is an RDD which we can apply map(), filter(), and reduce() transforms to. Suppose we wanted to know the five most common words ever written by William Shakespeare. We could solve this problem by accomplishing the following:
1. Create a similar map to our previous example 
2. Filter out all of the special characters 
3. Reducing the result 
4. Sort the result in descending order
5. Use the take() method to return the top five most common words. 

In [3]:
map_rdd = text_file_rdd.map(lambda x: (x.split(), 1))
map_rdd.take(5)

[(['From', 'fairest', 'creatures', 'we', 'desire', 'increase,'], 1),
 (['That', 'thereby', "beauty's", 'rose', 'might', 'never', 'die,'], 1),
 (['But', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease,'], 1),
 (['His', 'tender', 'heir', 'might', 'bear', 'his', 'memory:'], 1),
 (['But', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes,'], 1)]

Hmmm that doesn't look right, now does it? Instead of creating individual tokens, we've created a list of words that occurred for that given line. We need to flatten this result somehow so that when we map it, we get what we expect. Spark provides a special transformation for this called flatMap().

In [4]:
map_rdd = text_file_rdd.flatMap(lambda line: line.split())
map_rdd.take(10)

['From',
 'fairest',
 'creatures',
 'we',
 'desire',
 'increase,',
 'That',
 'thereby',
 "beauty's",
 'rose']

Much better. This is something we can definitely map.

In [5]:
words_rdd = map_rdd.filter(str.isalnum).map(lambda x: (x.lower(), 1))

And our reduction

In [6]:
results_rdd = words_rdd.reduceByKey(lambda x, y: x + y)
results_rdd.sortBy(lambda x: x[1], ascending=False).take(5)

[('the', 27458), ('and', 25916), ('i', 19540), ('to', 18656), ('of', 17877)]

And there we have it. The most commonly used word in William Shakespeare's extensive vocabulary is the word "the". That's... less than interesting. Let's take this result and introduce my favorite datastructure in Spark, the Dataframe.

### Dataframes
In Spark, a dataframe is simply a collection of RDD's that have been organized into columns. What makes this powerful is that we can now treat our data more like a table and write SQL-like queries to filter and transform the data. This isn't quite as performant as working on the raw RDD, but it is much more user friendly. Let's revisit the first couple of steps, and make a couple of minor changes.

In [7]:
map_rdd = text_file_rdd.flatMap(lambda line: line.split())
words_rdd = map_rdd.filter(str.isalnum).filter(lambda x: len(x) > 3).map(lambda x: (x.lower(), ))

Now, instead of reducing our key value pairs, we're going to create a Spark Session from our Spark Context, and then use this session to create our dataframe.

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
session = SparkSession(sc)
df = session.createDataFrame(words_rdd, ['word'])
df.createOrReplaceTempView("word_count")

When we create a dataframe, we specify the names of our columns as list, and then we register our new dataframe as a table we can query. In this case, I've chosen to name the column "word", and the table "word_count". Use this information to write a SQL query that answers the question, what are the top five most used words in Shakespeare's vocabulary? How about Shakespeare's top five least used words? Note that we've already filtered words that might be considered less interesting (ie words of fewer than 3 characters), so there's no need to write a convoluted WHERE class to filter out uninteresting words.

In [ ]:
most_used = session.sql("""YOUR ANSWER HERE""").take()

least_used = session.sql("""YOUR ANSWER HERE""").take()

In [ ]:
tests.part_e(most_used, least_used)